In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
data = pd.read_csv('spam.csv',encoding = "ISO-8859-1")
# Keeping only the neccessary columns
data = data[['v2','v1']]
# print(data)

                                                     v2    v1
0     Go until jurong point, crazy.. Available only ...   ham
1                         Ok lar... Joking wif u oni...   ham
2     Free entry in 2 a wkly comp to win FA Cup fina...  spam
3     U dun say so early hor... U c already then say...   ham
4     Nah I don't think he goes to usf, he lives aro...   ham
...                                                 ...   ...
5567  This is the 2nd time we have tried 2 contact u...  spam
5568              Will Ì_ b going to esplanade fr home?   ham
5569  Pity, * was in mood for that. So...any other s...   ham
5570  The guy did some bitching but I acted like i'd...   ham
5571                         Rofl. Its true to its name   ham

[5572 rows x 2 columns]


In [3]:
data['v2'] = data['v2'].apply(lambda x: x.lower())
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

print(data[data['v1'] == 'ham'].size)
print(data[data['v1'] == 'spam'].size)

9650
1494


In [0]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [5]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)
print(X)
X = pad_sequences(X)
print(X)
embed_dim = 128
lstm_out = 196

[[47, 445, 800, 717, 684, 64, 9, 1250, 90, 119, 361, 1251, 149, 1252, 67, 58, 137], [48, 316, 1403, 446, 6, 1830], [50, 466, 9, 22, 4, 755, 903, 1, 180, 1595, 1136, 630, 1831, 266, 71, 1595, 1, 1832, 1, 317, 466, 79, 385], [6, 233, 143, 24, 362, 6, 161, 144, 60, 143], [954, 2, 43, 97, 72, 467, 1, 904, 72, 1833, 206, 111, 447], [849, 120, 67, 1596, 41, 100, 186, 602, 21, 7, 39, 349, 86, 363, 56, 109, 386, 3, 42, 12, 15, 84, 1834, 48, 374, 1041, 1, 70, 306, 1], [207, 11, 603, 8, 25, 56, 1, 364, 35, 10, 108, 685, 10, 56], [73, 220, 13, 1137, 1404, 1404, 1835, 112, 100, 578, 73, 13, 955, 905, 12, 51, 1597, 956, 468, 1, 1042, 13, 221, 955, 905], [686, 73, 4, 801, 415, 222, 3, 17, 100, 416, 1, 1043, 140, 957, 1, 115, 16, 115, 387, 508, 718, 552, 64], [129, 13, 92, 1044, 802, 27, 124, 6, 85, 1138, 1, 469, 1, 5, 307, 529, 850, 35, 333, 12, 50, 16, 5, 92, 469, 1253, 50, 18], [23, 223, 32, 80, 213, 7, 2, 43, 69, 1, 269, 77, 40, 281, 1139, 214, 181, 169, 417, 91], [1836, 1, 180, 174, 46, 318, 1, 

In [0]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

In [7]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(3733, 152) (3733, 2)
(1839, 152) (1839, 2)


In [8]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)
model.save('spam_model.h5')
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 33s - loss: 0.1898 - accuracy: 0.9379
Epoch 2/5
 - 30s - loss: 0.0579 - accuracy: 0.9823
Epoch 3/5
 - 30s - loss: 0.0379 - accuracy: 0.9904
Epoch 4/5
 - 30s - loss: 0.0193 - accuracy: 0.9936
Epoch 5/5
 - 30s - loss: 0.0128 - accuracy: 0.9968
0.07110418857008426
0.9847743511199951
